# Exploring the results
## Output files
Let's have a look at the results. We will focus on the output from the second run as this will be the same as the first run but will also include the core gene alignment produced by MAFFT. We will start by looking at the most important output files and after this we will look at how you can query your pan genome, produce some plots and draw a simple tree form the core gene alignment.

Start by moving into the results directory:

In [ ]:
cd output_with_alignment

### summary_statistics.txt
The summary_statistics.txt file contains a summary of the number of genes in the core, pan and accessory genomes. It provides an overview of the genes and how frequently they occur in the input isolates. You should expect the total number of genes in this file to be about 1,000 genes per million bases of your genome. In this case, the genomes are around 2 mega bases, so we would expect a total number of genes to be in the order of 2,000. Let's have a look and see if this is the case.

In [ ]:
cat summary_statistics.txt

As you can see, we have around 2,500 which seems about right. If you get a lot fewer or too many genes than expected this could be an indication of some issue with your input data, for example contamination.

### gene_presence_absence.csv
The gene_presence_absence files lists each gene and which samples it is present in. The .csv file can be opened in Excel.

Let's have a look at the first ten lines of the file:

In [ ]:
head gene_presence_absence.csv

The columns are tab separated and contains the following information:

 1. The gene name, which is the most frequently occurring gene name from the sequences in the cluster. If there is no gene name, then it is given a generic unique name group_XXX.
 2. A non unique gene name, where sequences with the same gene name have ended up in different groups. It might be because of split genes, or miss annotation.
 3. Functional annotation. The most frequently occurring functional annotation from the cluster is used.
 4. Number of isolates represented in the cluster.
 5. Number of sequences in the cluster.
 6. Average number of sequences per isolate. This is normally 1. If this is greater than 1 then there is over clustering and the paralogs couldn’t be split.
 7. Genome fragment, where there is evidence at the contig level that the genes are linked.
 8. Order within fragment, combined with the genome fragment this gives an indication of the order of genes within the graph. In Excel, sort on Column G and H.
 9. Accessory Fragment is where core genes are excluded and there is evidence at contig level that the genes are linked.
 10. Accessory order with fragment, combined with the Accessory fragment this gives an indication of the order of genes within the accessory graph. In Excel, sort on columns I and J.
 11. Comments on the quality of the cluster. Miss predictions are noted, as are single genes on single contigs, which can be evidence of low level contamination.
 12. Minimum sequence length in nucleotides of the cluster.
 13. Maximum sequence length in nucleotides of the cluster.
 14. Average sequence length in nucleotides of the cluster.
 15. Presence and absence of genes in each sample, with the corresponding source Gene ID.

The .Rtab file contains a tab delimited binary matrix with the precence and abscence of each gene in each sample. This makes it easy to load into R using the read.table function. The first row is the header containing the name of each sample, and the first column contains the gene name. 1 indicates the gene is present in the sample and 0 indicates it is absent.

### pan_genome_reference.fa
This fasta file contains a single nucleotide sequence from each of the clusters in the pan genome. The name of each sequence is the source sequence ID followed by the cluster it came from. This file can be of use for reference guided assembly, whole genome MLST or for mapping raw reads to it.

### *.Rtab
### accessory_binary_genes.fa.newick
### accessory_graph.dot
### core_accessory_graph.dot
### clustered_proteins
### core_gene_alignment.aln
## query_pan_genome
## roary_plots.py
## Draw a tree form the core gene alignment
FastTree –nt –gtr core_gene_alignment.aln > my_tree.newick